In [1]:
import sys
sys.path.append('..')
import ScraperFC as sfc # import local scraperfc
import traceback
from tqdm.auto import tqdm
import pandas as pd

In [2]:
# # from shared_test_functions import get_random_league_seasons

# # samples = get_random_league_seasons('Oddsportal', 3)

# op = sfc.Oddsportal()
# # fbref = sfc.FBRef()
# counts = dict()
# try:
#     # for year, league in tqdm(samples):
#     league = 'EPL'
#     for year in tqdm(range(2004,2023)):
#         # print('-'*80)
#         # print(year, league)
#         # fbref_links = fbref.get_match_links(year,league)
#         op_links = op.get_match_links(year, league)
#         counts[year] = len(op_links)
#         # break
# except:
#     traceback.print_exc()
# finally:
#     op.close()
#     # fbref.close()

In [3]:
op = sfc.Oddsportal()
try:
    # match_link = links[np.random.randint(0, num_links+1)]
    # match_link = 'https://oddsportal.com/football/england/premier-league-2005-2006/chelsea-birmingham-be9APe6e' # no O/U odds
    # match_link = 'https://www.oddsportal.com/football/england/premier-league/west-ham-southampton-tfnoGQAd' # weird date format
    # match_link = 'https://www.oddsportal.com/football/england/premier-league/aston-villa-bournemouth-Kdi8EDZi/' # everything should work
    # match_df = op.scrape_match(match_link)
    # moneyline_df = op.get_1X2odds_from_match(match_link)
    # over_under_df = op.get_OUodds_from_match(match_link)

    # season_df = op.scrape_season_odds(2020, 'EPL')
except:
    traceback.print_exc()
finally:
    op.close()

2020 EPL:   0%|          | 0/380 [00:00<?, ?it/s]